In [2]:
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
!pip install --no-deps packaging ninja einops flash-attn trl peft accelerate bitsandbytes
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Looking in indexes: https://download.pytorch.org/whl/cu121
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-g67q4s0i/unsloth_5a6c356d5cc64514b99c2e7b71a68b1f
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-g67q4s0i/unsloth_5a6c356d5cc64514b99c2e7b71a68b1f
  Resolved https://github.com/unslothai/unsloth.git to commit a395211e02334e53c653b53156da05bb8440d42a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import torch
import os
import json
import pandas as pd
from datasets import Dataset, DatasetDict
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [4]:
huggingface_user = "GaoDalie"
dataset_name = "ScienceQA"

class Llama3InstructDataset:
    def __init__(self, data):
        self.data = data
        self.prompts = []
        self.create_prompts()

    def create_prompt(self, row):
        prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>{row['instruction']}<|eot_id|><|start_header_id|>user<|end_header_id|>{row['question']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>{row['correct_answer']}<|eot_id|>"""
        return prompt

    def create_prompts(self):
        for row in self.data:
            prompt = self.create_prompt(row)
            self.prompts.append(prompt)

    def get_dataset(self):
        df = pd.DataFrame({'prompt': self.prompts})
        return df

def create_dataset_hf(dataset):
    dataset.reset_index(drop=True, inplace=True)
    return DatasetDict({"train": Dataset.from_pandas(dataset)})

if __name__ == "__main__":
    with open('/content/dataset_QA.json', 'r') as f:
        data = json.load(f)

    dataset = Llama3InstructDataset(data)
    df = dataset.get_dataset()

    processed_data_path = 'processed_data'
    os.makedirs(processed_data_path, exist_ok=True)

    llama3_dataset = create_dataset_hf(df)
    llama3_dataset.save_to_disk(os.path.join(processed_data_path, "llama3_dataset"))
    # llama3_dataset.push_to_hub(f"{huggingface_user}/{dataset_name}")

Saving the dataset (0/1 shards):   0%|          | 0/101 [00:00<?, ? examples/s]

In [5]:
# Defining the configuration for the base model, LoRA and training
config = {
    "hugging_face_username":huggingface_user,
    "model_config": {
        "base_model":"unsloth/llama-3-8b-Instruct-bnb-4bit", # The base model
        "finetuned_model":"llama-3-8b-Instruct-bnb-4bit-gaodalie-1-demo", # The finetuned model
        "max_seq_length": 2048, # The maximum sequence length
        "dtype":torch.float16, # The data type
        "load_in_4bit": True, # Load the model in 4-bit
    },
    "lora_config": {
      "r": 16, # The number of LoRA layers 8, 16, 32, 64
      "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"], # The target modules
      "lora_alpha":16, # The alpha value for LoRA
      "lora_dropout":0, # The dropout value for LoRA
      "bias":"none", # The bias for LoRA
      "use_gradient_checkpointing":True, # Use gradient checkpointing
      "use_rslora":False, # Use RSLora
      "use_dora":False, # Use DoRa
      "loftq_config":None # The LoFTQ configuration
    },
    "training_dataset":{
        "name":f"{huggingface_user}/{dataset_name}", # The dataset name(huggingface/datasets)
        "split":"train", # The dataset split
        "input_field":"prompt", # The input field
    },
    "training_config": {
        "per_device_train_batch_size": 2, # The batch size
        "gradient_accumulation_steps": 4, # The gradient accumulation steps
        "warmup_steps": 5, # The warmup steps
        "max_steps":0, # The maximum steps (0 if the epochs are defined)
        "num_train_epochs": 12, # The number of training epochs(0 if the maximum steps are defined)
        "learning_rate": 2e-4, # The learning rate
        "fp16": not torch.cuda.is_bf16_supported(),  # The fp16
        "bf16": torch.cuda.is_bf16_supported(), # The bf16
        "logging_steps": 1, # The logging steps
        "optim" :"adamw_8bit", # The optimizer
        "weight_decay" : 0.01,  # The weight decay
        "lr_scheduler_type": "linear", # The learning rate scheduler
        "seed" : 42, # The seed
        "output_dir" : "outputs", # The output directory
    }
}

In [6]:
# Loading the model and the tokinizer for the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = config.get("model_config").get("base_model"),
    max_seq_length = config.get("model_config").get("max_seq_length"),
    dtype = config.get("model_config").get("dtype"),
    load_in_4bit = config.get("model_config").get("load_in_4bit"),
)

# Setup for QLoRA/LoRA peft of the base model
model = FastLanguageModel.get_peft_model(
    model,
    r = config.get("lora_config").get("r"),
    target_modules = config.get("lora_config").get("target_modules"),
    lora_alpha = config.get("lora_config").get("lora_alpha"),
    lora_dropout = config.get("lora_config").get("lora_dropout"),
    bias = config.get("lora_config").get("bias"),
    use_gradient_checkpointing = config.get("lora_config").get("use_gradient_checkpointing"),
    random_state = 42,
    use_rslora = config.get("lora_config").get("use_rslora"),
    use_dora = config.get("lora_config").get("use_dora"),
    loftq_config = config.get("lora_config").get("loftq_config"),
)

# Loading the training dataset
dataset_train = load_dataset(config.get("training_dataset").get("name"), split = config.get("training_dataset").get("split"))

# Setting up the trainer for the model
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    dataset_text_field = config.get("training_dataset").get("input_field"),
    max_seq_length = config.get("model_config").get("max_seq_length"),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = config.get("training_config").get("per_device_train_batch_size"),
        gradient_accumulation_steps = config.get("training_config").get("gradient_accumulation_steps"),
        warmup_steps = config.get("training_config").get("warmup_steps"),
        max_steps = config.get("training_config").get("max_steps"),
        num_train_epochs= config.get("training_config").get("num_train_epochs"),
        learning_rate = config.get("training_config").get("learning_rate"),
        fp16 = config.get("training_config").get("fp16"),
        bf16 = config.get("training_config").get("bf16"),
        logging_steps = config.get("training_config").get("logging_steps"),
        optim = config.get("training_config").get("optim"),
        weight_decay = config.get("training_config").get("weight_decay"),
        lr_scheduler_type = config.get("training_config").get("lr_scheduler_type"),
        seed = 42,
        output_dir = config.get("training_config").get("output_dir"),
    ),
)

==((====))==  Unsloth 2024.10.0: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.10.0 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 101 | Num Epochs = 12
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 144
 "-____-"     Number of trainable parameters = 41,943,040
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
1,3.023200
2,3.864300
3,3.473600
4,3.097100
5,2.246700
6,2.244500
7,2.035400
8,1.743100
9,1.477200
10,1.282500


In [8]:
with open("trainer_stats.json", "w") as f:
    json.dump(trainer_stats, f, indent=4)

In [9]:
model.save_pretrained_gguf(config.get("model_config").get("finetuned_model"), tokenizer, quantization_method = "q4_k_m")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.85 out of 12.67 RAM for saving.


100%|██████████| 32/32 [01:13<00:00,  2.28s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving llama-3-8b-Instruct-bnb-4bit-gaodalie-1-demo/pytorch_model-00001-of-00004.bin...
Unsloth: Saving llama-3-8b-Instruct-bnb-4bit-gaodalie-1-demo/pytorch_model-00002-of-00004.bin...
Unsloth: Saving llama-3-8b-Instruct-bnb-4bit-gaodalie-1-demo/pytorch_model-00003-of-00004.bin...
Unsloth: Saving llama-3-8b-Instruct-bnb-4bit-gaodalie-1-demo/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at llama-3-8b-Instruct-bnb-4bit-gaodalie-1-demo into f16 GGUF format.
The output location will be /content/llama-3-8b-Instruct-bnb-4bit-gaodalie-1-demo/unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: llama-3-8b-Instruct-bnb-4bit-gaodalie-1-demo
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin'
INFO:hf-to-gguf:token_embd.w

In [10]:
# Loading the fine-tuned model and the tokenizer for inference
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = config.get("model_config").get("finetuned_model"),
        max_seq_length = config.get("model_config").get("max_seq_length"),
        dtype = config.get("model_config").get("dtype"),
        load_in_4bit = config.get("model_config").get("load_in_4bit"),
    )

# Using FastLanguageModel for fast inference
FastLanguageModel.for_inference(model)

system_prompt = f"You are a scientist. I will provide a question, and you will respond with the result based on your scientific knowledge. Answer the question using the provided context. If the question cannot be answered with the given information, respond with 'I don't know."

# Tokenizing the input and generating the output
prompt = input('TYPE PROMPT TO LLAMA3: ')
inputs = tokenizer(
[
    f"<|start_header_id|>system<|end_header_id|>{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>{prompt}<|end_header_id|>"
], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs, skip_special_tokens = True)

==((====))==  Unsloth 2024.10.0: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

TYPE PROMPT TO LLAMA3: Most of the chemical reactions in the body are facilitated by what?


["systemYou are a scientist. I will provide a question, and you will respond with the result based on your scientific knowledge. Answer the question using the provided context. If the question cannot be answered with the given information, respond with 'I don't know.userMost of the chemical reactions in the body are facilitated by what?assistant\n\nMost of the chemical reactions in the body are facilitated by enzymes. Enzymes are biological molecules, typically proteins, that speed up chemical reactions by lowering the activation energy required for the reaction to occur. They do this by binding to the reactants, positioning them in a way that allows the reaction to take place more easily, and stabilizing the transition state. This allows the reaction to proceed faster and more efficiently than it would without the enzyme."]

In [17]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 10.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.1-cp310-cp310-linux_x86_64.whl size=3485348 sha256=593510506234e03753a6b9e9f66f74c65dc245ec47de8c414a9bd21c3c78002f
  Stored in directory: /root/.cache/pip/wheels/f8/b0/a2/f47d952aec7ab061b9e2a345e23a1e1e137beb7891259e3d0c
Successfully built llama-cpp-python


In [18]:
from llama_cpp import Llama

In [19]:
# GLOBAL VARIABLES
my_model_path = "/content/llama-3-8b-Instruct-bnb-4bit-gaodalie-1-demo/unsloth.Q4_K_M.gguf"
CONTEXT_SIZE = 512

In [20]:
llama_model = Llama(model_path=my_model_path,
                    n_ctx=CONTEXT_SIZE)

llama_model_loader: loaded meta data with 29 key-value pairs and 291 tensors from /content/llama-3-8b-Instruct-bnb-4bit-gaodalie-1-demo/unsloth.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3 8b Instruct Bnb 4bit
llama_model_loader: - kv   3:                            general.version str              = 1
llama_model_loader: - kv   4:                       general.organization str              = Unsloth
llama_model_loader: - kv   5:                           general.finetune str              = Instruct-bnb-4bit
llama_model_loader: - kv   6:                           general.basename str              = llama-

In [24]:
def generate_text_from_prompt(user_prompt,
                             max_tokens = 300,
                             temperature = 0.3,
                             top_p = 0.1,
                             echo = True,
                             stop = ["Q", "\n"]):




   # Define the parameters
   model_output = llama_model(
       user_prompt,
       max_tokens=max_tokens,
       temperature=temperature,
       top_p=top_p,
       echo=echo,
       stop=stop,
   )


   return model_output

In [25]:
if __name__ == "__main__":


   my_prompt = "In order to create food, what do photosynthetic protists use?"


   zephyr_model_response = generate_text_from_prompt(my_prompt)


   print(zephyr_model_response)

Llama.generate: 1 prefix-match hit, remaining 15 prompt tokens to eval
llama_perf_context_print:        load time =    7394.25 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    15 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    24 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   23901.80 ms /    39 tokens


{'id': 'cmpl-4627586e-53b7-4c3a-b875-59d84e229949', 'object': 'text_completion', 'created': 1729180781, 'model': '/content/llama-3-8b-Instruct-bnb-4bit-gaodalie-1-demo/unsloth.Q4_K_M.gguf', 'choices': [{'text': 'In order to create food, what do photosynthetic protists use? A. water and carbon dioxide B. nitrogen and oxygen C. minerals and sunlight D. nitrates and phosphates', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 16, 'completion_tokens': 25, 'total_tokens': 41}}


In [26]:
if __name__ == "__main__":


   my_prompt = "Which radio frequency should you listen to if you want less noise?"


   zephyr_model_response = generate_text_from_prompt(my_prompt)


   print(zephyr_model_response)

Llama.generate: 1 prefix-match hit, remaining 13 prompt tokens to eval
llama_perf_context_print:        load time =    7394.25 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    13 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    14 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   15226.31 ms /    27 tokens


{'id': 'cmpl-c17e92ab-d45a-44a9-a853-031d236da876', 'object': 'text_completion', 'created': 1729180854, 'model': '/content/llama-3-8b-Instruct-bnb-4bit-gaodalie-1-demo/unsloth.Q4_K_M.gguf', 'choices': [{'text': 'Which radio frequency should you listen to if you want less noise? A) FM B) AM C) HD Radio D) Satellite Radio', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 14, 'completion_tokens': 15, 'total_tokens': 29}}


In [27]:
!pip install langchain_community langchain_core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully 

In [37]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate


template = """ You are a scientist. I will provide a question, and you will respond with the result based on your scientific knowledge. Answer the question using the provided context. If the question cannot be answered with the given information, respond with 'I don't know

Question: {question}

Answer: {Final answer}."""
prompt = PromptTemplate.from_template(template)
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

my_model_path = "/content/llama-3-8b-Instruct-bnb-4bit-gaodalie-1-demo/unsloth.Q4_K_M.gguf"

# 初始化 LLM 设置
llm = LlamaCpp(
    model_path=my_model_path,
    temperature=0.75,
    max_tokens=300,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,
)

question = """
Question: The formation of an amalgam allows the metal to react with what?
"""
llm.invoke(question)


llama_model_loader: loaded meta data with 29 key-value pairs and 291 tensors from /content/llama-3-8b-Instruct-bnb-4bit-gaodalie-1-demo/unsloth.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3 8b Instruct Bnb 4bit
llama_model_loader: - kv   3:                            general.version str              = 1
llama_model_loader: - kv   4:                       general.organization str              = Unsloth
llama_model_loader: - kv   5:                           general.finetune str              = Instruct-bnb-4bit
llama_model_loader: - kv   6:                           general.basename str              = llama-

A. Air
B. Water
C. Mercury
D. The surrounding tooth material

Answer: D. The surrounding tooth material
The formation of an amalgam allows the metal to react with the surrounding tooth material, which is mostly dentin. The mercury in the amalgam reacts with the copper in the alloy to form a stable compound that prevents further corrosion. The resulting amalgam is strong and durable, making it well suited for use in dental restorations such as fillings and inlays. In addition, amalgams are also used in crowns, bridges, and dentures. Amalgams have been used in dentistry for over 150 years, and they continue to be a popular choice among dentists due to their durability, strength, and ease of use. However, some people may experience adverse effects from the mercury in amalgams, such as oral or gastrointestinal symptoms, or neurological problems. As a result, some dentists may choose to avoid using amalgams in their practice, or they may take steps to minimize exposure to mercury for themse

llama_perf_context_print:        load time =    8696.75 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    18 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   299 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  227416.14 ms /   317 tokens


'A. Air\nB. Water\nC. Mercury\nD. The surrounding tooth material\n\nAnswer: D. The surrounding tooth material\nThe formation of an amalgam allows the metal to react with the surrounding tooth material, which is mostly dentin. The mercury in the amalgam reacts with the copper in the alloy to form a stable compound that prevents further corrosion. The resulting amalgam is strong and durable, making it well suited for use in dental restorations such as fillings and inlays. In addition, amalgams are also used in crowns, bridges, and dentures. Amalgams have been used in dentistry for over 150 years, and they continue to be a popular choice among dentists due to their durability, strength, and ease of use. However, some people may experience adverse effects from the mercury in amalgams, such as oral or gastrointestinal symptoms, or neurological problems. As a result, some dentists may choose to avoid using amalgams in their practice, or they may take steps to minimize exposure to mercury for